# Learning Python (Day 51)

# Python Flask - User Authentication

User authentication ensures that users are who they claim to be and restricts access to resources based on their identity. Here’s a detailed explanation with examples to help understand the implementation in a Flask application.

**_Code - https://github.com/HeyBuddy-NSK/nkblog_**

## 1. User Model

The `User` model represents the user in the database. It includes fields like `id`, `username`, `email`, and `password_hash`.

**Example:**
```python
from werkzeug.security import generate_password_hash, check_password_hash
from flask_login import UserMixin

class User(UserMixin, db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(64), unique=True, nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    password_hash = db.Column(db.String(128), nullable=False)

    def set_password(self, password):
        self.password_hash = generate_password_hash(password)

    def check_password(self, password):
        return check_password_hash(self.password_hash, password)

    def __repr__(self):
        return f'<User {self.username}>'
```

**Explanation:**
- `UserMixin` adds default implementations for methods required by Flask-Login.
- `set_password` hashes the password and stores it.
- `check_password` verifies if the provided password matches the stored hash.

## 2. Password Hashing

To securely store passwords, we use hashing functions.

**Example:**
```python
from werkzeug.security import generate_password_hash, check_password_hash

# Hash a password
hashed_password = generate_password_hash('mysecretpassword')

# Check a password
is_correct_password = check_password_hash(hashed_password, 'mysecretpassword')
```

**Explanation:**
- `generate_password_hash` creates a hashed version of the password.
- `check_password_hash` checks if a given password matches the hash.

## 3. User Registration

A registration form is created using Flask-WTF, and the form data is validated to create new users.

**Form Definition:**
```python
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField
from wtforms.validators import DataRequired, Email, EqualTo

class RegistrationForm(FlaskForm):
    username = StringField('Username', validators=[DataRequired()])
    email = StringField('Email', validators=[DataRequired(), Email()])
    password = PasswordField('Password', validators=[DataRequired()])
    password2 = PasswordField('Repeat Password', validators=[DataRequired(), EqualTo('password')])
    submit = SubmitField('Register')
```

**View Function:**
```python
@app.route('/register', methods=['GET', 'POST'])
def register():
    form = RegistrationForm()
    if form.validate_on_submit():
        user = User(username=form.username.data, email=form.email.data)
        user.set_password(form.password.data)
        db.session.add(user)
        db.session.commit()
        flash('Congratulations, you are now a registered user!')
        return redirect(url_for('login'))
    return render_template('register.html', form=form)
```

**Explanation:**
- The `RegistrationForm` class defines the form fields and their validation.
- The `register` route handles form submission, user creation, and redirection.

## 4. User Login

A login form is implemented to authenticate users and establish a session.

**Form Definition:**
```python
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField
from wtforms.validators import DataRequired

class LoginForm(FlaskForm):
    username = StringField('Username', validators=[DataRequired()])
    password = PasswordField('Password', validators=[DataRequired()])
    remember_me = BooleanField('Remember Me')
    submit = SubmitField('Login')
```

**View Function:**
```python
from flask_login import login_user, logout_user, login_required, current_user

@app.route('/login', methods=['GET', 'POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user is None or not user.check_password(form.password.data):
            flash('Invalid username or password')
            return redirect(url_for('login'))
        login_user(user, remember=form.remember_me.data)
        return redirect(url_for('index'))
    return render_template('login.html', form=form)
```

**Explanation:**
- The `LoginForm` class defines the form fields and their validation.
- The `login` route handles form submission, user authentication, and session establishment.

## 5. Flask-Login Integration

Flask-Login provides user session management. Initialization and user loader callback are required.

**Initialization:**
```python
from flask_login import LoginManager

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'
```

**User Loader:**
```python
@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))
```

**Explanation:**
- `LoginManager` initializes Flask-Login.
- `user_loader` callback is used to load a user from the database by ID.

## 6. Protecting Routes

Use the `@login_required` decorator to restrict access to authenticated users only.

**Example:**
```python
@app.route('/protected')
@login_required
def protected():
    return 'Logged in as: ' + current_user.username
```

**Explanation:**
- The `@login_required` decorator ensures that only logged-in users can access the route.

## 7. Logging Out

Implementing logout functionality to end the user session.

**Example:**
```python
@app.route('/logout')
@login_required
def logout():
    logout_user()
    flash('You have been logged out.')
    return redirect(url_for('index'))
```

**Explanation:**
- The `logout` route logs out the user and redirects to the home page.

## 8. Remember Me

Adding "Remember Me" functionality to keep users logged in between sessions.

**Login Form:**
```python
class LoginForm(FlaskForm):
    username = StringField('Username', validators=[DataRequired()])
    password = PasswordField('Password', validators=[DataRequired()])
    remember_me = BooleanField('Remember Me')
    submit = SubmitField('Login')
```

**Login View:**
```python
@app.route('/login', methods=['GET', 'POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user is None or not user.check_password(form.password.data):
            flash('Invalid username or password')
            return redirect(url_for('login'))
        login_user(user, remember=form.remember_me.data)
        return redirect(url_for('index'))
    return render_template('login.html', form=form)
```

**Explanation:**
- The `remember_me` field in the form captures whether the user wants to be remembered.
- The `login_user` function handles the "Remember Me" functionality.

## 9. Email Confirmation

Send confirmation emails during registration to verify user email addresses.

**Email Sending Function:**
```python
def send_confirmation_email(user):
    token = user.generate_confirmation_token()
    send_mail(user.email, 'Confirm Your Account', 'auth/email/confirm', user=user, token=token)
```

**Email Confirmation View:**
```python
@app.route('/confirm/<token>')
@login_required
def confirm(token):
    if current_user.confirmed:
        return redirect(url_for('index'))
    if current_user.confirm(token):
        db.session.commit()
        flash('You have confirmed your account. Thanks!')
    else:
        flash('The confirmation link is invalid or has expired.')
    return redirect(url_for('index'))
```

**Generating and Confirming Tokens:**
```python
from itsdangerous import TimedJSONWebSignatureSerializer as Serializer
from flask import current_app

class User(UserMixin, db.Model):
    # other fields...

    def generate_confirmation_token(self, expiration=3600):
        s = Serializer(current_app.config['SECRET_KEY'], expiration)
        return s.dumps({'confirm': self.id}).decode('utf-8')

    def confirm(self, token):
        s = Serializer(current_app.config['SECRET_KEY'])
        try:
            data = s.loads(token)
        except:
            return False
        if data.get('confirm') != self.id:
            return False
        self.confirmed = True
        db.session.add(self)
        return True
```

**Explanation:**
- `generate_confirmation_token` creates a token for email confirmation.
- `confirm` verifies the token and activates the user account.
- `send_confirmation_email` sends the email with the token.
- The `confirm` route verifies the token when the user clicks the link in the email.